In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import sys
import os
sys.path.append("/content/drive/My Drive/GoogleColab/pytorch3d_packages")

In [6]:
# --- Imports -------------------------------------------------------------------
import os, glob, numpy as np, zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# --- Paths ---------------------------------------------------------------------
zip_path = "/content/drive/MyDrive/GoogleColab/ShapeNetCore.zip"
extract_path = "/content/ShapeNetCore/ShapeNetCore"

# Extract zip if not already extracted
if not os.path.exists(extract_path):
    print(f"Extracting {zip_path} to {extract_path} ...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/ShapeNetCore")
    print("Extraction complete.")

# Verify extraction
print("Top-level classes:", os.listdir(extract_path))

# --- Binvox Reader -------------------------------------------------------------
def read_binvox(filepath):
    """Read .binvox file and return voxel grid as numpy array."""
    with open(filepath, "rb") as f:
        line = f.readline().decode("ascii").strip()
        if not line.startswith("#binvox"):
            raise IOError("Not a binvox file")
        dims, translate, scale = None, None, None
        while True:
            line = f.readline().decode("ascii").strip()
            if line.startswith("dim"):
                dims = list(map(int, line.split()[1:]))
            elif line.startswith("translate"):
                translate = list(map(float, line.split()[1:]))
            elif line.startswith("scale"):
                scale = float(line.split()[1])
            elif line.startswith("data"):
                break
        if dims is None:
            raise IOError("Missing dimensions in binvox")

        # Convert compressed .binvox data into a 3D voxel grid
        raw_data = np.frombuffer(f.read(), dtype=np.uint8)
        values, counts = raw_data[0::2], raw_data[1::2]
        voxels = np.repeat(values, counts).astype(np.bool_)
        return voxels.reshape(dims)

# --- Dataset -------------------------------------------------------------------
class ShapeNetVoxDataset(Dataset):
    def __init__(self, root_dir, variant="surface"):
        """
        variant: "surface" or "solid"
        """
        self.root_dir = root_dir
        self.items, self.classes = [], []
        self.variant = variant

        class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        class_dirs.sort()
        self.classes = class_dirs
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        for c in self.classes:
            class_dir = os.path.join(root_dir, c)
            binvox_files = glob.glob(os.path.join(class_dir, "**", "*.binvox"), recursive=True)
            for f in binvox_files:
                if variant in f:
                    self.items.append((f, self.class_to_idx[c]))
            print(f"[DEBUG] Class {c}: found {len(self.items)} items so far")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        filepath, label = self.items[idx]
        vox = read_binvox(filepath)
        vox = torch.from_numpy(vox).float().unsqueeze(0)
        vox = F.interpolate(vox.unsqueeze(0), size=(32,32,32), mode="nearest").squeeze(0)
        return vox, label

# --- Model ---------------------------------------------------------------------
class Small3DCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(128 * 4 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# --- Training Loop -------------------------------------------------------------
def run_training(dataset_root=extract_path, epochs=5, batch_size=8, lr=1e-3, variant="surface"):

    dataset = ShapeNetVoxDataset(dataset_root, variant=variant)
    if len(dataset) == 0:
        raise RuntimeError("Dataset is empty — check if .binvox files exist!")

    n_classes = len(dataset.classes)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Small3DCNN(num_classes=n_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for vox, labels in train_loader:
            vox, labels = vox.to(device), torch.tensor(labels).to(device)
            optimizer.zero_grad()
            outputs = model(vox)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * vox.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        train_acc = 100. * correct / total

        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for vox, labels in val_loader:
                vox, labels = vox.to(device), torch.tensor(labels).to(device)
                outputs = model(vox)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * vox.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()
        val_acc = 100. * correct / total

        print(f"Epoch {epoch+1}/{epochs} "
              f"- TrainLoss: {train_loss/len(train_loader.dataset):.4f}, TrainAcc: {train_acc:.2f}% "
              f"- ValLoss: {val_loss/len(val_loader.dataset):.4f}, ValAcc: {val_acc:.2f}%")

    return model

# --- Run Training --------------------------------------------------------------
model = run_training(epochs=5, batch_size=8, lr=1e-3, variant="surface")


Top-level classes: ['02808440', '03642806', '02992529', '03211117', '03046257']
[DEBUG] Class 02808440: found 856 items so far
[DEBUG] Class 02992529: found 1686 items so far
[DEBUG] Class 03046257: found 2337 items so far
[DEBUG] Class 03211117: found 3427 items so far
[DEBUG] Class 03642806: found 3887 items so far


/tmp/ipython-input-2243258162.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vox, labels = vox.to(device), torch.tensor(labels).to(device)
/tmp/ipython-input-2243258162.py:141: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vox, labels = vox.to(device), torch.tensor(labels).to(device)


Epoch 1/5 - TrainLoss: 0.5122, TrainAcc: 83.95% - ValLoss: 0.3953, ValAcc: 86.12%
Epoch 2/5 - TrainLoss: 0.2642, TrainAcc: 91.61% - ValLoss: 0.2795, ValAcc: 89.97%
Epoch 3/5 - TrainLoss: 0.1888, TrainAcc: 93.86% - ValLoss: 0.2133, ValAcc: 92.93%
Epoch 4/5 - TrainLoss: 0.1352, TrainAcc: 95.72% - ValLoss: 0.2118, ValAcc: 94.34%
Epoch 5/5 - TrainLoss: 0.0998, TrainAcc: 97.14% - ValLoss: 0.2836, ValAcc: 92.29%
